In [28]:
! pip install gdown 
! gdown 1z28MmoZmBd46FRF4hmFkmLIFemqxg5C8
! unzip -o jigsaw-toxic-comment-classification-challenge.zip
! rm -f jigsaw-toxic-comment-classification-challenge.zip
! unzip -o train.csv.zip -d /content/ 
! unzip -o test.csv.zip -d /content/ 
#https://drive.google.com/file/d/1z28MmoZmBd46FRF4hmFkmLIFemqxg5C8/view?usp=sharing

zsh:1: command not found: pip
zsh:1: command not found: gdown
unzip:  cannot find or open jigsaw-toxic-comment-classification-challenge.zip, jigsaw-toxic-comment-classification-challenge.zip.zip or jigsaw-toxic-comment-classification-challenge.zip.ZIP.
Archive:  train.csv.zip
checkdir:  cannot create extraction directory: /content
           Read-only file system
Archive:  test.csv.zip
checkdir:  cannot create extraction directory: /content
           Read-only file system


In [29]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/arkeshkalathiya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/arkeshkalathiya/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arkeshkalathiya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/arkeshkalathiya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
import pandas as pd
import re
import os
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from pprint import pprint
import numpy as np
from gensim.utils import simple_preprocess
import joblib
from IPython.display import clear_output

from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import KeyedVectors
from nltk import word_tokenize

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import multilabel_confusion_matrix, classification_report, plot_roc_curve

import gensim.downloader as trained_models
import os.path as path

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, MultiLabelBinarizer

from sklearn.model_selection import RandomizedSearchCV

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV, GridSearchCV

import warnings
warnings.filterwarnings('ignore')  # supress warnings


In [ ]:
#### extract all the zip files to unprocessed folder

! unzip -o ./train.csv.zip -d ./unprocessed
! unzip -o ./test.csv.zip -d ./unprocessed
! unzip -o ./test_labels.csv.zip -d ./unprocessed

clear_output(False)

In [ ]:
englishStopWords = list(stopwords.words('english'))
englishStopWords.extend(['utc','talk'])
englishStopWords = set(englishStopWords)
lemmatizer = WordNetLemmatizer()

nDAY = r'(?:[0-3]?\d)'  # day can be from 1 to 31 with a leading zero 
nMNTH = r'(?:11|12|10|0?[1-9])' # month can be 1 to 12 with a leading zero
nYR = r'(?:(?:19|20)\d\d)'  # I've restricted the year to being in 20th or 21st century on the basis 
                            # that people doon't generally use all number format for old dates, but write them out 
nDELIM = r'(?:[\/\-\._])?'  # 
NUM_DATE = f"""
    (?P<num_date>
        (?:^|\D) # new bit here
        (?:
        # YYYY-MM-DD
        (?:{nYR}(?P<delim1>[\/\-\._]?){nMNTH}(?P=delim1){nDAY})
        |
        # YYYY-DD-MM
        (?:{nYR}(?P<delim2>[\/\-\._]?){nDAY}(?P=delim2){nMNTH})
        |
        # DD-MM-YYYY
        (?:{nDAY}(?P<delim3>[\/\-\._]?){nMNTH}(?P=delim3){nYR})
        |
        # MM-DD-YYYY
        (?:{nMNTH}(?P<delim4>[\/\-\._]?){nDAY}(?P=delim4){nYR})
        )
        (?:\D|$) # new bit here
    )"""
DAY = r"""
(?:
    # search 1st 2nd 3rd etc, or first second third
    (?:[23]?1st|2{1,2}nd|\d{1,2}th|2?3rd|first|second|third|fourth|fifth|sixth|seventh|eighth|nineth)
    |
    # or just a number, but without a leading zero
    (?:[123]?\d)
)"""
MONTH = r'(?:january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|mar|apr|may|jun|jul|aug|sep|sept|oct|nov|dec)'
YEAR = r"""(?:(?:[12]?\d|')?\d\d)"""
DELIM = r'(?:\s*(?:[\s\.\-\\/,]|(?:of))\s*)'

YEAR_4D = r"""(?:[12]\d\d\d)"""
DATE_PATTERN = f"""(?P<wordy_date>
    # non word character or start of string
    (?:^|\W)
        (?:
            # match various combinations of year month and day 
            (?:
                # 4 digit year
                (?:{YEAR_4D}{DELIM})?
                    (?:
                    # Day - Month
                    (?:{DAY}{DELIM}{MONTH})
                    |
                    # Month - Day
                    (?:{MONTH}{DELIM}{DAY})
                    )
                # 2 or 4 digit year
                (?:{DELIM}{YEAR})?
            )
            |
            # Month - Year (2 or 3 digit)
            (?:{MONTH}{DELIM}{YEAR})
        )
    # non-word character or end of string
    (?:$|\W)
)"""

TIME = r"""(?:
(?:
# first number should be 0 - 59 with optional leading zero.
[012345]?\d
# second number is the same following a colon
:[012345]\d
)
# next we add our optional seconds number in the same format
(?::[012345]\d)?
# and finally add optional am or pm possibly with . and spaces
(?:\s*(?:a|p)\.?m\.?)?
)"""

COMBINED = f"""(?P<combined>
    (?:
        # time followed by date, or date followed by time
        {TIME}?{DATE_PATTERN}{TIME}?
        |
        # or as above but with the numeric version of the date
        {TIME}?{NUM_DATE}{TIME}?
    ) 
    # or a time on its own
    |
    (?:{TIME})
)"""

regexDate = re.compile(COMBINED, re.IGNORECASE | re.VERBOSE | re.UNICODE)


def removeNumber(comment):
    return re.sub(r'\d+',"",comment)

def removeDoubleDots(comment):
    return re.sub(r'\.{2,}',"",comment)

def removeDoubleSpace(comment):
    return re.sub(r'[ ]{2,}'," ",comment)

def removePunctuations(comment):
    return re.sub(r'[^\w\s]','',comment)

def removeNewLine(comment):
    return re.sub(r'\n'," ",comment)

def removeURLs(comment):
    return re.sub(r'^https?:\/\/.*[\r\n]*', '', comment, flags=re.MULTILINE)

def removeDate(comment):
    return regexDate.sub(' ',comment)

def removeRepeatingWords(comment):
    return re.sub(r'(\w+)( \1)+', r'\1', comment, flags=re.IGNORECASE)

def lemmatize(comment):
    tokens = [lemmatizer.lemmatize(word) for word in nltk.word_tokenize(comment) if word not in englishStopWords]
    return " ".join(tokens)


def processComment(comment):
    comment = removeURLs(comment)
    comment = removeDate(comment)
    comment = removeNumber(comment)
    comment = removeNewLine(comment)
    comment = removeDoubleSpace(comment)
    comment = removeDoubleDots(comment)
    comment = removePunctuations(comment)
    comment = lemmatize(comment)
    comment = removeRepeatingWords(comment)
    return comment.lower()    




In [ ]:
class TextCleaner(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        return X.apply(lambda x : processComment(x))

In [ ]:
class ParagraphVectorizer(BaseEstimator, TransformerMixin):

    def __init__(self):
        dataset = './processed/w2vEmbeddings.model'
        self.wv = KeyedVectors.load(dataset, mmap='r')
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        finalVector = []
        for paragraph in X:
            pVector = []
            tokens = word_tokenize(paragraph)
            for token in tokens:
                if(self.wv.__contains__(token)):
                    pVector.append(self.wv[token])
            if len(pVector) > 0:
                pVector = np.average(pVector,axis=0).tolist()
            else:
                pVector = np.zeros(100)
            finalVector.append(pVector)
        return pd.DataFrame(finalVector)

In [ ]:
# create pre processed data directory to optimize the preprocessing


def mergeTrainCommentAndLabels(dfComment, dfLabels):
    for index, row in dfComment.iterrows():
        index = row.name
        targetRow = dfLabels.iloc[index]
        if row.id != targetRow.id:
            print("Not matching")
        dfComment.at[index, 'toxic'] = int(targetRow.toxic)
        dfComment.at[index, 'severe_toxic'] = int(targetRow.severe_toxic)
        dfComment.at[index, 'obscene'] = int(targetRow.obscene)
        dfComment.at[index, 'threat'] = int(targetRow.threat)
        dfComment.at[index, 'insult'] = int(targetRow.insult)
        dfComment.at[index, 'identity_hate'] = int(targetRow.identity_hate)
        
    finalDf = dfComment.copy(deep=True)
    finalDf = finalDf.drop(finalDf[finalDf.toxic == -1].index)
    finalDf['comment_text'] = finalDf['comment_text'].apply( lambda x: processComment(x))
    return finalDf

if not os.path.exists('./processed'):
    os.makedirs('./processed')

if not os.path.exists('./processed/train.csv'):
    dataset = pd.read_csv('./unprocessed/train.csv')
    dataset['comment_text'] = dataset['comment_text'].apply(
        lambda x: processComment(x))
    dataset.to_csv('./processed/train.csv')

if not os.path.exists('./processed/test.csv'):
    x = pd.read_csv('./unprocessed/test.csv')
    y = pd.read_csv('./unprocessed/test_labels.csv')
    finalDf = mergeTrainCommentAndLabels(x,y)
    finalDf.to_csv('./processed/test.csv')


In [ ]:
trainData = pd.read_csv('./processed/train.csv')
testData = pd.read_csv('./processed/test.csv')
trainData = trainData.dropna()
testData = testData.dropna()

In [ ]:
trainData.shape

(159343, 9)

In [ ]:
testData.shape

(63875, 9)

In [ ]:
trainData.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0000997932d777bf,explanation why edits made username hardcore m...,0,0,0,0,0,0
1,1,000103f0d9cfb60f,daww he match background colour im seemingly s...,0,0,0,0,0,0
2,2,000113f07ec002fd,hey man im really trying edit war its guy cons...,0,0,0,0,0,0
3,3,0001b41b1c6bb37e,more i cant make real suggestion improvement i...,0,0,0,0,0,0
4,4,0001d958c54c6e35,you sir hero any chance remember page thats,0,0,0,0,0,0


In [ ]:
testData.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,5,0001ea8717f6de06,thank understanding i think highly would rever...,0.0,0.0,0.0,0.0,0.0,0.0
1,7,000247e83dcc1211,dear god site horrible,0.0,0.0,0.0,0.0,0.0,0.0
2,11,0002f87b16116a7f,somebody invariably try add religion reallyou ...,0.0,0.0,0.0,0.0,0.0,0.0
3,13,0003e1cccfd5a40a,it say right is type the type institutioneeded...,0.0,0.0,0.0,0.0,0.0,0.0
4,14,00059ace3e3e9a53,before adding new product list make surelevant...,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Generate the word2vec embeddings

from gensim.models import Word2Vec

if not os.path.exists('./processed/w2vEmbeddings.model'):
    word2vec = Word2Vec(
        sentences=trainData['comment_text'].apply(
            lambda x: nltk.word_tokenize(str(x))
        ), 
        vector_size=100, 
        window=5, 
        min_count=1, 
        workers=10
    )
    word2vec.wv.save('./processed/w2vEmbeddings.model')


In [ ]:
countVectorizerParams = {
    # 'vectorizer__max_df': (0.5, 1.0),
    # 'vectorizer__ngram_range': ((1, 1), (1, 2)),
}

tfidfVectorizerParams = {
    # 'vectorizer__strip_accents' : ('ascii', 'unicode'),
}

logisticRegressionParams = {
    # 'classifier__solver':['newton-cg', 'saga'],
}

mnbClassifierParams = {
    # 'classifier__alpha' : [.2,.5,1],
}

rfClassifierParams = {
    # 'classifier__max_depth' : [30,40],
}

dtClassifierParams = {
    # 'classifier__splitter' : ["best", "random"]
}


In [ ]:
tuples = {
    "transformers": [
        {
            "title": "TfIDF Vectorizer",
            "tuples": [
                ('vectorizer', TfidfVectorizer(analyzer='word'))
            ],
            "params": {
                **tfidfVectorizerParams,
            }
        },
        {
            "title": "Count Vectorizer",
            "tuples": [
                ('vectorizer', CountVectorizer(analyzer='word')),
            ],
            "params": {
                **countVectorizerParams,
            }
        },
        {
            "title": "Paragraph Vectorizer",
            "tuples": [
                ('vectorizer', ParagraphVectorizer()),
            ],
            "params": {}
        }
    ],
    "classifiers": [
        # {
        #     "title" : "Multinomial Naive Bayes",
        #     "tuples" : [
        #         ('scaler',MinMaxScaler()),
        #         ('classifier',MultiOutputClassifier(MultinomialNB()))
        #     ],
        #     "params" : {
        #         **mnbClassifierParams
        #     }
        # },
        {
            "title": "Logistic Regression",
            "tuples": [
                ('classifier', MultiOutputClassifier(LogisticRegression()))
            ],
            "params": {
                **logisticRegressionParams
            }
        },
        {
            "title": "Random Forest Classifier",
            "tuples": [
                ('classifier', MultiOutputClassifier(RandomForestClassifier()))
            ],
            "params": {
                **rfClassifierParams
            }
        },
        {
            "title": "Decision Tree Classifier",
            "tuples": [
                ('classifier', MultiOutputClassifier(DecisionTreeClassifier()))
            ],
            "params": {
                **dtClassifierParams
            }
        }
    ]
}


In [ ]:
def generateIdForRun(transformerName, classifierName):
    name = " ".join([transformerName, classifierName])
    codeName = re.sub(r'[ ]+', "_", name)
    codeName = codeName.lower()
    return codeName


In [ ]:

labels = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
best_models =[]

for transformer in tuples["transformers"]:
    transformerName = transformer["title"]
    transformerTuple = transformer["tuples"]
    transformerParams = transformer["params"]
    for classifier in tuples["classifiers"]:
        classifierName = classifier["title"]
        classifierTuples = classifier["tuples"]
        classifierParams = classifier["params"]

        print(f'=== Running {transformerName} + {classifierName}')

        finalParams = {
            **transformerParams,
            **classifierParams
        }

        pipeline = Pipeline(
            transformerTuple
            + classifierTuples
        )

        searchCV = GridSearchCV(
            pipeline, 
            finalParams, 
            cv=3,
            n_jobs=32, 
            verbose=1,
            error_score='raise'
        )

        name = " ".join([transformerName,classifierName])
        idForRun = generateIdForRun(transformerName,classifierName)
        

        if not os.path.exists(f'./dumps'):
            os.makedirs('./dumps')

        if os.path.exists(f'./dumps/{idForRun}.bin'):
            print(f"{name} is already trained")
            continue

        dataset = trainData.copy()

        searchCV.fit(dataset['comment_text'],dataset[labels])

        bestEstimator = searchCV.best_estimator_
        bestScore = searchCV.best_score_
        bestParams = searchCV.best_params_

        best_models.append((f'{transformerName} & {classifierName}',bestEstimator,bestScore,bestParams))

        print(f"{name} score : {bestScore}")

        joblib.dump(bestEstimator,f'./dumps/{idForRun}.bin')

=== Running TfIDF Vectorizer + Logistic Regression
TfIDF Vectorizer Logistic Regression is already trained
=== Running TfIDF Vectorizer + Random Forest Classifier
TfIDF Vectorizer Random Forest Classifier is already trained
=== Running TfIDF Vectorizer + Decision Tree Classifier
TfIDF Vectorizer Decision Tree Classifier is already trained
=== Running Count Vectorizer + Logistic Regression
Count Vectorizer Logistic Regression is already trained
=== Running Count Vectorizer + Random Forest Classifier
Count Vectorizer Random Forest Classifier is already trained
=== Running Count Vectorizer + Decision Tree Classifier
Count Vectorizer Decision Tree Classifier is already trained
=== Running Paragraph Vectorizer + Logistic Regression
Paragraph Vectorizer Logistic Regression is already trained
=== Running Paragraph Vectorizer + Random Forest Classifier
Paragraph Vectorizer Random Forest Classifier is already trained
=== Running Paragraph Vectorizer + Decision Tree Classifier
Paragraph Vectoriz

In [ ]:
def explain(name,score,params):
    print("Search for CV "+name)
    print("Best score: %0.3f" % score)
    best_parameters = params
    for param_name in sorted(params.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

for (name,estimator,score,param) in best_models:
    explain(name,score,param)


In [34]:
import seaborn as sn
import matplotlib.pyplot as plt

for transformer in tuples["transformers"]:
    transformerName = transformer["title"]
    for classifier in tuples["classifiers"]:
        classifierName = classifier["title"]

        name = f"{transformerName} {classifierName}"
        title = f"{transformerName} & {classifierName}"
        idForRun = generateIdForRun(transformerName, classifierName)
        outputDir = f'./outputs/{name}'

        print(f"Generating graph for {name}")

        if not os.path.exists(f'./dumps/{idForRun}.bin'):
            continue

        if not os.path.exists(outputDir):
            os.makedirs(outputDir)

        model = joblib.load(f'./dumps/{idForRun}.bin')

        
        plot_roc_curve(model,testData.comment_text,testData[labels])
        plt.show()
        break

        # predictions = pd.DataFrame(model.predict(
        #     testData.comment_text), columns=labels)

        # cnfMatList = multilabel_confusion_matrix(testData[labels], predictions)
        # report = classification_report(testData[labels], predictions,target_names=labels)
        
        

        # for index, label in enumerate(labels):
        #     columns = ["Not "+label,label]
        #     df = pd.DataFrame(cnfMatList[index], columns, columns)
        #     ax = plt.axes()
        #     sn.heatmap(df, ax=ax, annot=True, cmap='Blues',fmt='d')
        #     ax.set_title(title,pad=15)
        #     plt.savefig(f'{outputDir}/{label}.png')
        #     plt.close()


Generating graph for TfIDF Vectorizer Logistic Regression


AttributeError: 'list' object has no attribute 'ndim'